In [15]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.metrics import *
from collections import Counter
from pre_process import PreProcess


In [16]:
import pandas as pd

data = pd.read_csv('./spam.csv')
print(data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [17]:
print(type(data["Message"]))
data["Message"] = PreProcess.preProcess(data["Message"])
print(data["Message"][0])

<class 'pandas.core.series.Series'>
go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   


In [18]:
print(data["Message"][0])
print(data["Message"].size)
for i in range(0, data["Message"].size):
  data["Message"][i] = PreProcess.standardize(data["Message"][i])
print(data["Message"][0])

go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   
5572
go jurong point crazy available bugis n great world la e buffet cine get amore wat


In [19]:
data["Label"] = 0
for i in range(0, data["Category"].size):
  data["Label"][i] = PreProcess.transformLabel(data["Category"][i])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29824\2858425811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Label"][i] = PreProcess.transformLabel(data["Category"][i])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29824\2858425811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Label"][i] = PreProcess.transformLabel(data["Category"][i])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_29824\2858425811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [20]:
X = data["Message"]
Y = data["Label"]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)
print(X_train, X_test, Y_train, Y_test)

1114                          good movie ok leave hourish
3589    free give otherwise nalla adi entey nattil kittum
3095             emigrate something ok maybe bite hopeful
1012                            get home babe still awake
3320                                    kay since already
                              ...                        
4931    hi sexychat girl wait text text great night ch...
3264                                  u gonna get deus ex
1653    ur chance win cash every wk txt action c www m...
2607                    r u sam p eachother meet go house
2732                  mm feel sleepy today shall get dear
Name: Message, Length: 4457, dtype: object 4456    storm msg wen u lift phne u say hello u knw wt...
690           please call immediately urgent message wait
944     also sorta blow couple time recently id rather...
3768                            sir goodmorning free call
1189             come alive good correct good look figure
                             

In [21]:
vocabulary = PreProcess.createVocabulary(X_train)
print(len(vocabulary))
print(vocabulary)

1320
['good', 'movie', 'ok', 'leave', 'free', 'give', 'otherwise', 'something', 'maybe', 'bite', 'get', 'home', 'babe', 'still', 'awake', 'since', 'already', 'workin', 'job', 'say', 'matter', 'mind', 'oh', 'yeah', 'diet', 'fly', 'sorry', 'thing', 'may', 'pub', 'late', 'ill', 'call', 'even', 'idea', 'dear', 'room', 'look', 'daddy', 'want', 'eat', 'night', 'long', 'chennai', 'surely', 'pick', 'u', 'da', 'car', 'park', 'okay', 'soon', 'best', 'yar', 'else', 'thk', 'sort', 'funny', 'yup', 'izzit', 'rain', 'co', 'e', 'mrt', 'c', 'outside', 'way', 'ur', 'pop', 'ask', 'bout', 'around', 'wen', 'girl', 'come', 'talk', 'g', 'x', 'cool', 'tyler', 'take', 'gonna', 'buy', 'drop', 'place', 'tonight', 'order', 'enough', 'work', 'time', 'village', 'near', 'house', 'would', 'wake', 'pls', 'family', 'hot', 'sun', 'strange', 'reason', 'go', 'invite', 'actually', 'wait', 'serious', 'whats', 'hill', 'hope', 'great', 'day', 'r', 'fine', 'busy', 'though', 'many', 'company', 'tell', 'honey', 'find', 'much', '

In [22]:
from joblib import dump
dump(vocabulary, "vocabulary")

['vocabulary']

In [23]:
for text in X_train:
  text = PreProcess.filter(text, vocabulary)

for text in X_test:
  text = PreProcess.filter(text, vocabulary)
 
print(X_train)

1114                          good movie ok leave hourish
3589    free give otherwise nalla adi entey nattil kittum
3095             emigrate something ok maybe bite hopeful
1012                            get home babe still awake
3320                                    kay since already
                              ...                        
4931    hi sexychat girl wait text text great night ch...
3264                                  u gonna get deus ex
1653    ur chance win cash every wk txt action c www m...
2607                    r u sam p eachother meet go house
2732                  mm feel sleepy today shall get dear
Name: Message, Length: 4457, dtype: object


In [24]:
cv = CountVectorizer()
print(cv)
cv.fit_transform(vocabulary)
features_CV = cv.transform(X_train)
features_test_cv = cv.transform(X_test)

print(features_CV)

CountVectorizer()
  (0, 447)	1
  (0, 602)	1
  (0, 715)	1
  (0, 774)	1
  (1, 407)	1
  (1, 439)	1
  (1, 795)	1
  (2, 114)	1
  (2, 672)	1
  (2, 774)	1
  (2, 1019)	1
  (3, 77)	1
  (3, 81)	1
  (3, 434)	1
  (3, 502)	1
  (3, 1048)	1
  (4, 33)	1
  (4, 997)	1
  (5, 434)	1
  (5, 561)	1
  (5, 1267)	1
  (6, 667)	2
  (6, 687)	1
  (6, 948)	2
  (7, 284)	1
  :	:
  (4452, 1051)	2
  (4452, 1101)	2
  (4452, 1210)	1
  (4453, 352)	1
  (4453, 434)	1
  (4453, 446)	1
  (4454, 167)	1
  (4454, 175)	1
  (4454, 251)	1
  (4454, 348)	1
  (4454, 1163)	1
  (4454, 1165)	1
  (4454, 1183)	1
  (4454, 1251)	1
  (4454, 1259)	2
  (4454, 1278)	1
  (4455, 442)	1
  (4455, 512)	1
  (4455, 676)	1
  (4456, 268)	1
  (4456, 378)	1
  (4456, 434)	1
  (4456, 693)	1
  (4456, 975)	1
  (4456, 1130)	1


In [25]:
model_svm = svm.SVC( C = 100, gamma = 0.01)
model_svm.fit(features_CV, Y_train)

SVC(C=100, gamma=0.01)

In [26]:
Y_preds_svm = model_svm.predict(features_test_cv)
print('SVM:')
print('Accuracy: ', accuracy_score(Y_test, Y_preds_svm)*100)
print('Precision: ', precision_score(Y_test, Y_preds_svm)*100)
print('Recall: ', recall_score(Y_test, Y_preds_svm)*100)
print('Confusion Matrix: ', confusion_matrix(Y_test, Y_preds_svm))
print('********************************************************')

SVM:
Accuracy:  98.11659192825111
Precision:  94.83870967741936
Recall:  91.875
Confusion Matrix:  [[947   8]
 [ 13 147]]
********************************************************


In [27]:
from joblib import dump
dump(model_svm, 'model_svm')

['model_svm']